<a href="https://colab.research.google.com/github/KyunghoonJeon/TensorflowIntro/blob/main/chap5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/3minten

/content/drive/MyDrive/3minten


In [4]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
import numpy as np

data=np.loadtxt('./data.csv', delimiter=',',skiprows=1,
                unpack=True, dtype='float32')

x_data=np.transpose(data[0:2])
y_data=np.transpose(data[2:])

In [6]:
global_step=tf.Variable(0, trainable=False, name='global_step')

In [7]:
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

W1=tf.Variable(tf.random_uniform([2,10], -1., 1.)) 
L1=tf.nn.relu(tf.matmul(X,W1))

W2=tf.Variable(tf.random_uniform([10,20], -1., 1.))
L2=tf.nn.relu(tf.matmul(L1,W2))

W3=tf.Variable(tf.random_uniform([20,3], -1., 1.))
model=tf.matmul(L2, W3)

cost=tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,
                                               logits=model))

optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
train_op=optimizer.minimize(cost, global_step=global_step)

In [8]:
sess=tf.Session()
saver=tf.train.Saver(tf.global_variables())

In [10]:
ckpt=tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [55]:
for step in range(2):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})

    print('Step: %d, ' % sess.run(global_step),
        'Cost: %.3f'% sess.run(cost, feed_dict={X:x_data, Y:y_data}))

Step: 47,  Cost: 0.552
Step: 48,  Cost: 0.552


In [33]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

Instructions for updating:
Use standard file APIs to delete files with this prefix.


'./model/dnn.ckpt-12'

In [37]:
prediction=tf.argmax(model,1)
target=tf.argmax(Y,1)
print('예측값:', sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y:y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [38]:
is_correct=tf.equal(prediction, target)
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data}))

정확도: 100.00
